# Industry Return Predictability: A Machine Learning Approach
This study replicate the paper writen by Rapach, Strauss, Tu, and Zhou(RSTZ).

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib

import requests
# To load Fama and French data (we could also use wrds)
pd.core.common.is_list_like = pd.api.types.is_list_like 
import pandas_datareader

import statsmodels.api as sm
import scipy.stats

import warnings
warnings.simplefilter('ignore')

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [129]:
import sklearn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, mean_squared_error, explained_variance_score, r2_score, accuracy_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

### Data
30 industry monthly return (from December 1959 to December 2016) from Ken French website

In [4]:
# Ken French 30 industries
data = pandas_datareader.famafrench.FamaFrenchReader('30_Industry_Portfolios',start='1959-12', end='2020-12')
data   = data.read()[0] # Monthly data
data.columns = [i.strip() for i in data.columns]
data

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm,Servs,BusEq,Paper,Trans,Whlsl,Rtail,Meals,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1959-12,2.35,0.69,-2.68,1.98,7.63,1.01,2.21,-1.63,3.42,1.08,...,4.31,-1.62,1.01,1.80,2.37,0.04,2.75,3.05,0.49,-6.16
1960-01,-4.16,-5.38,-1.72,1.54,-5.14,-7.51,-8.20,-6.35,-9.70,-4.44,...,0.95,-5.85,-7.60,-9.08,-3.98,-5.00,-5.76,-9.75,-4.35,-3.65
1960-02,3.64,-1.85,2.56,4.52,2.68,9.60,1.73,0.27,-0.45,0.61,...,8.36,9.42,5.38,3.29,-0.65,1.71,4.29,2.10,-0.69,6.61
1960-03,-1.32,-2.59,0.17,-0.30,2.53,-0.21,-2.24,1.61,-2.40,-6.44,...,0.14,0.04,3.69,-2.08,-4.64,-1.02,0.22,-3.53,0.40,-2.08
1960-04,1.36,-1.97,1.54,6.65,-0.98,-1.08,0.40,1.68,-5.34,-0.91,...,-1.05,7.33,1.96,0.60,-1.94,0.64,-0.34,9.05,-0.45,0.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02,-8.83,-7.43,-6.71,-5.05,-6.61,-8.48,-9.36,-5.63,-10.12,-8.16,...,-5.94,-5.55,-9.16,-7.38,-10.96,-9.38,-6.16,-9.02,-10.60,-8.57
2020-03,-8.73,-9.88,-6.75,-16.42,-23.08,-6.84,-18.53,-5.31,-18.37,-35.96,...,-13.39,-11.52,-8.80,-9.76,-16.47,-17.86,-4.43,-21.67,-20.03,-13.46
2020-04,7.17,11.01,1.97,15.17,8.94,10.32,10.56,13.10,18.43,13.15,...,9.64,16.11,13.96,9.73,8.52,12.68,18.64,18.43,11.71,3.98


In [5]:
# one-month treasury bill return
data1 = pandas_datareader.famafrench.FamaFrenchReader('F-F_Research_Data_Factors',start='1959-12', end='2020-12')
data1 = data1.read()[0] # Monthly data
rf = data1['RF']
rf

Date
1959-12   0.34
1960-01   0.33
1960-02   0.29
1960-03   0.35
1960-04   0.19
          ... 
2020-02   0.12
2020-03   0.12
2020-04   0.00
2020-05   0.01
2020-06   0.01
Freq: M, Name: RF, Length: 727, dtype: float64

### Table 1 - Summary statistics, industry portfolio excess returns, 1959-12 to 2016-12

In [6]:
# excess return
er = data.sub(rf, axis=0)

# summary statistics table
sst = pd.DataFrame(index = er.columns)
sst['Ann. mean (%)'] = er.mean(axis=0)*12
sst['Ann. volatility (%)'] = er.std(axis=0)*np.sqrt(12)
sst['Minimum (%)'] = er.min(axis=0)
sst['Maximum (%)'] = er.max(axis=0)
sst['Ann. Sharpe ratio'] = sst['Ann. mean (%)'] / sst['Ann. volatility (%)']

In [7]:
sst

,Ann. mean (%),Ann. volatility (%),Minimum (%),Maximum (%),Ann. Sharpe ratio
Food,7.94,14.93,-18.15,19.89,0.53
Beer,8.48,17.49,-20.19,25.51,0.49
Smoke,10.87,21.04,-25.32,32.38,0.52
Games,8.99,24.84,-33.40,34.52,0.36
Books,5.66,20.30,-26.56,33.13,0.28
Hshld,6.90,16.34,-22.24,18.22,0.42
Clths,8.45,22.17,-31.50,31.79,0.38
Hlth,8.05,16.97,-21.06,29.01,0.47
Chems,6.04,19.31,-28.60,21.68,0.31
Txtls,6.99,24.93,-36.08,59.03,0.28


### Table 2 - OLS post-LASSO predictive regression estimation results, 1960:01 - 2016:12

In [8]:
# data process
X = er.copy().shift(1)
Y = er.copy()
X = X.drop(X.index[0])
Y = Y.drop(Y.index[0])

In [9]:
# RUN REGRESSION
res = pd.DataFrame(0,columns = X.columns, index = X.columns)
r_square = pd.Series(index = X.columns)
nrows, npreds = X.shape

# use aic lasso to eliminate overfitting problem
# only keep the most related independent variables
# then run regression on these independent variable
for industry in X.columns:
    y = Y[industry]
    model_aic = LassoLarsIC(criterion='aic')
    model_aic.fit(X,y)
    predcols = [i for i in range(npreds) if model_aic.coef_[i] !=0]

    if predcols != []:
        x = X.iloc[:,predcols]
        model_ols = LinearRegression()
        model_ols.fit(x, y)
        y_pred = model_ols.predict(x)
        r_square.loc[industry] = (100 * r2_score(y, y_pred))
        for i in range(len(model_ols.coef_)):
            res.loc[X.columns[predcols[i]], industry] = model_ols.coef_[i]

In [10]:
# coeeficient table
res

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm,Servs,BusEq,Paper,Trans,Whlsl,Rtail,Meals,Fin,Other
Food,0,0.23,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Beer,0,-0.12,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,...,-0.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Smoke,0,0.00,0.00,-0.07,0.00,0.00,0.00,0.00,0,0.00,...,0.00,0.00,-0.15,0.00,0.00,-0.07,0.00,0.00,0.00,0.00
Games,0,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Books,0,0.00,0.00,0.11,0.00,0.00,0.00,0.06,0,0.00,...,0.00,0.00,0.06,0.00,0.00,0.09,0.00,0.04,0.00,0.00
Hshld,0,0.00,-0.06,0.00,0.00,0.00,0.00,0.00,0,-0.17,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Clths,0,0.06,0.00,0.06,0.03,0.09,0.07,0.03,0,0.07,...,0.00,0.00,0.00,0.06,0.03,0.00,0.00,0.10,0.00,0.07
Hlth,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Chems,0,0.00,0.00,0.00,0.00,0.00,0.13,0.00,0,0.22,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Txtls,0,0.00,0.11,0.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [11]:
# r-squared
r_square
print(np.mean(r_square))

3.798049791227396


### Table 3 - Industry-rotation portfolio performance, 1970:01 - 2016:12

In [127]:
# train the model
def fit_predict(X, Y, model_selection, Xpredict = None, model_type = LinearRegression()):
    '''
    Fit regression and make prediction for one time step
    model_selection: 'prevailing', 'OLS', 'OLS Lasso','OLS Lasso Subset'
    Xpredict: if model_selection is prevailing, Xpredict is none
    '''
    res = pd.Series(index = X.columns)
    model = model_type
    
    for industry in X.columns:
        y = Y[industry]
        
        if model_selection == 'OLS Lasso Subset':
            model_aic = LassoLarsIC(criterion='aic')
            model_aic.fit(X,y)
            predcols = [i for i in range(npreds) if model_aic.coef_[i] !=0]

            if predcols != []:
                x = X.iloc[:,predcols]
                model.fit(x, y)
                y_pred = model.predict([Xpredict[predcols]])
                
            else:
                y_pred = X[industry].mean()
            
        elif model_selection == 'OLS':
            model.fit(X, y)
            y_pred = model.predict([Xpredict])
            
        elif model_selection == 'OLS Lasso':
            model_aic = LassoLarsIC(criterion='aic')
            model_aic.fit(X,y)
            y_pred = model_aic.predict([Xpredict])
            
        elif model_selection == 'SVM':
            clf = SVR(kernel='rbf')
            #grid_values = {'C': [0.01, 0.1, 1, 10, 100],'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100]}
            #grid_clf = GridSearchCV(clf, param_grid = grid_values, scoring = 'r2')
            clf.fit(X, y)
            y_pred = clf.predict([Xpredict])
            
        elif model_selection == 'RFT':
            clf = RandomForestRegressor(max_depth = 5)
            #grid_values = {'max_depth': list(range(6)),'learning_rate':[0.01,0.05,0.1,0.5,1]}
            #grid_clf = GridSearchCV(clf, param_grid = grid_values, scoring = 'r2')
            clf.fit(X, y)
            y_pred = clf.predict([Xpredict])
            
        else:
            y_pred = X[industry].mean()
                
        res.loc[industry] = y_pred
        
    return res

In [116]:
# malke predictions for all industry 
def predictions(X, Y, predict_start, model_selection):
    '''
    Return a dataframe with predicted returns for all industry for all period
    predict_start: from which date the prediciton start
    '''
    forecast_period = X.loc[predict_start:].index
    Ypredict = pd.DataFrame(columns = X.columns, index = forecast_period)
    
    for i in range(len(forecast_period)):
        time_step = forecast_period[i]
        prev_time_step = time_step-1
        Xsub = X.loc[:prev_time_step]
        Ysub = Y.loc[:prev_time_step]

        if model_selection == 'prevailing':
            res = fit_predict(X = Xsub, Y = Ysub, model_selection = model_selection)
        else:
            res = fit_predict(X = Xsub, Y = Ysub, Xpredict = X.loc[prev_time_step], model_selection = model_selection)
        
        Ypredict.loc[time_step] = res
        
    return Ypredict

In [80]:
Ypredict = predictions(X, Y, '1970-01',model_selection='OLS Lasso')

In [81]:
Ypredict_nolasso = predictions(X, Y, '1970-01',model_selection='OLS')

In [82]:
Ypredict_prev = predictions(X, Y, '1970-01',model_selection='prevailing')

In [83]:
Ypredict_subset = predictions(X, Y, '1970-01',model_selection='OLS Lasso Subset')

In [124]:
Ypredict_svm = predictions(X, Y, '1970-01',model_selection='SVM')

In [36]:
def long_short_portfolio(Ypredict, Y, quantile = 0):
    Ypredict = Ypredict.astype('float64')
    long_short_label = pd.DataFrame(index = Ypredict.index)
    long_short_portfolio = pd.DataFrame(index = Ypredict.index, columns = ['long_rets','short_rets'])
    real_returns = Y.loc[Ypredict.index]
    if quantile == 0:
        long_short_label['long'] = Ypredict.idxmax(axis = 'columns')
        long_short_label['short'] = Ypredict.idxmin(axis = 'columns')

        for date in long_short_label.index:
            long_short_portfolio.loc[date, 'long_rets'] = real_returns.loc[date, long_short_label.loc[date, 'long']]
            long_short_portfolio.loc[date, 'short_rets'] = real_returns.loc[date, long_short_label.loc[date, 'short']]
    else:
        n = int(Ypredict.shape[1]/quantile)
        for date in long_short_label.index:
            rets = Ypredict.loc[date].sort_values(ascending = False)
            long_short_portfolio.loc[date, 'long_rets'] = real_returns.loc[date, rets.index[:n]].mean()
            long_short_portfolio.loc[date, 'short_rets'] = real_returns.loc[date, rets.index[-n:]].mean()          

    long_short_portfolio['rets'] = long_short_portfolio['long_rets'] - long_short_portfolio['short_rets']
    return long_short_portfolio['rets']

In [84]:
long_short_rets = long_short_portfolio(Ypredict, Y, quantile = 3)

In [85]:
long_short_rets_nolasso = long_short_portfolio(Ypredict_nolasso, Y, quantile = 3)

In [86]:
long_short_rets_prev = long_short_portfolio(Ypredict_prev, Y, quantile = 3)

In [87]:
long_short_rets_subset = long_short_portfolio(Ypredict_subset, Y, quantile = 3)

In [125]:
long_short_rets_svm = long_short_portfolio(Ypredict_svm, Y, quantile = 3)

In [ ]:
long_short_rets_rft = long_short_portfolio(Ypredict_rft, Y, quantile = 3)

In [40]:
def summary_statistics_table(df):
    sst = pd.Series()
    sst['Ann. mean (%)'] = df.mean(axis=0)*12
    sst['Ann. volatility (%)'] = df.std(axis=0)*np.sqrt(12)
    sst['Minimum (%)'] = df.min(axis=0)
    sst['Maximum (%)'] = df.max(axis=0)
    sst['Ann. Sharpe ratio'] = sst['Ann. mean (%)'] / sst['Ann. volatility (%)']
    return sst

In [88]:
summary_statistics_table(long_short_rets)

Ann. mean (%)           1.26
Ann. volatility (%)     9.25
Minimum (%)           -11.11
Maximum (%)             9.58
Ann. Sharpe ratio       0.14
dtype: float64

In [89]:
summary_statistics_table(long_short_rets_nolasso)

Ann. mean (%)           3.67
Ann. volatility (%)     9.00
Minimum (%)           -10.62
Maximum (%)            11.33
Ann. Sharpe ratio       0.41
dtype: float64

In [90]:
summary_statistics_table(long_short_rets_prev)

Ann. mean (%)          -0.91
Ann. volatility (%)     8.48
Minimum (%)           -19.72
Maximum (%)            11.94
Ann. Sharpe ratio      -0.11
dtype: float64

In [91]:
summary_statistics_table(long_short_rets_subset)

Ann. mean (%)           1.26
Ann. volatility (%)     8.91
Minimum (%)           -12.39
Maximum (%)             8.29
Ann. Sharpe ratio       0.14
dtype: float64

In [126]:
summary_statistics_table(long_short_rets_svm)

Ann. mean (%)           1.75
Ann. volatility (%)     8.15
Minimum (%)           -12.16
Maximum (%)            10.05
Ann. Sharpe ratio       0.22
dtype: float64

In [44]:
def long_only_portfolio(Ypredict, Y, quantile = 0):
    Ypredict = Ypredict.astype('float64')
    long_label = pd.DataFrame(index = Ypredict.index, columns = ['long'])
    long_portfolio = pd.DataFrame(index = Ypredict.index, columns = ['long_rets'])
    real_returns = Y.loc[Ypredict.index]
    if quantile == 0:
        long_label['long'] = Ypredict.idxmax(axis = 'columns')

        for date in long_short_label.index:
            long_portfolio.loc[date, 'long_rets'] = real_returns.loc[date, long_label.loc[date, 'long']]        
    else:
        n = int(Ypredict.shape[1]/quantile)
        for date in long_label.index:
            rets = Ypredict.loc[date].sort_values(ascending = False)
            long_portfolio.loc[date, 'long_rets'] = real_returns.loc[date, rets.index[:n]].mean()         

    return long_portfolio['long_rets']

In [100]:
long_rets_nolasso = long_only_portfolio(Ypredict_nolasso, Y, quantile = 3)

In [107]:
date = '2008-01'

In [108]:
summary_statistics_table(long_rets_nolasso.loc[date:])

Ann. mean (%)           8.87
Ann. volatility (%)    18.49
Minimum (%)           -20.12
Maximum (%)            24.25
Ann. Sharpe ratio       0.48
dtype: float64

In [48]:
summary_statistics_table(Y.loc[Ypredict.index].loc[date:].mean(axis=1))

Ann. mean (%)         -17.22
Ann. volatility (%)    37.85
Minimum (%)           -18.00
Maximum (%)            13.12
Ann. Sharpe ratio      -0.45
dtype: float64

***

## Traing data period
Instead of using all the previous data, use 120-month training data, since the latest data can capture the most recent relationship between industries

In [92]:
# malke predictions for all industry with rolling training data
def predictions_w_rolling_training_data(X, Y, predict_start, model_selection, rolling_window):
    '''
    Return a dataframe with predicted returns for all industry for all period
    predict_start: from which date the prediciton start
    '''
    forecast_period = X.loc[predict_start:].index
    Ypredict = pd.DataFrame(columns = X.columns, index = forecast_period)
    
    for i in range(len(forecast_period)):
        time_step = forecast_period[i]
        prev_time_step = time_step - 1
        rolling_time_step = time_step - rolling_window
        Xsub = X.loc[rolling_time_step:prev_time_step]
        Ysub = Y.loc[rolling_time_step:prev_time_step]

        if model_selection == 'prevailing':
            res = fit_predict(X = Xsub, Y = Ysub, model_selection = model_selection)
        else:
            res = fit_predict(X = Xsub, Y = Ysub, Xpredict = X.loc[prev_time_step], model_selection = model_selection)
        
        Ypredict.loc[time_step] = res
        
    return Ypredict

In [93]:
Ypredict_nolasso_rolling = predictions_w_rolling_training_data(X, Y, '1972-01',model_selection='OLS',rolling_window=100)

In [94]:
long_short_rets_nolasso_rolling = long_short_portfolio(Ypredict_nolasso_rolling, Y, quantile = 3)

In [97]:
long_rets_nolasso_rolling = long_only_portfolio(Ypredict_nolasso_rolling, Y, quantile = 3)

In [109]:
summary_statistics_table(long_rets_nolasso_rolling.loc[date:])

Ann. mean (%)           6.04
Ann. volatility (%)    18.16
Minimum (%)           -18.50
Maximum (%)            20.02
Ann. Sharpe ratio       0.33
dtype: float64